In [1]:
import requests
import pandas as pd
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

In [2]:
c_id = getpass.getpass()

········


In [3]:
c_secret = getpass.getpass()

········


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,client_secret=c_secret))

In [5]:
popvortex_100 = pd.read_csv('C:/Users/ebour/Documents/()_Ironhack DA Bootcamp/()_Labs/6_1_lab-web-scraping-single-page/popvortex_100.csv')

In [6]:
suggestion_database = pd.read_csv('clusterized_data.csv')

In [7]:
suggestion_database

,track_name,artist_name,cluster
0,The Joker And The Queen (feat. Taylor Swift),Ed Sheeran,6
1,The Joker And The Queen (feat. Taylor Swift),Taylor Swift,6
2,Zo Zijn,sor,1
3,Options (feat. 24kGoldn),Internet Money,0
4,Options (feat. 24kGoldn),24kGoldn,0
...,...,...,...
9512,Eu Sou a Diva Que Você Quer Copiar,Valesca Popozuda,1
9513,Tô Mal,César Menotti & Fabiano,2
9514,A Paz Desse Amor,Paula Fernandes,6
9515,Eu Ligo Pra Você,Zé Neto & Cristiano,7


# MVP

In [8]:
import pickle
filename = 'spotify_cluster_model.sav'
cluster_model = pickle.load(open(filename, 'rb'))

In [9]:
filename_scaler = 'spotify_scaler.sav'
scaler = pickle.load(open(filename_scaler, 'rb'))

In [57]:
def song_advice():
    song_input = input('Please fill in the name of a song you like : ')
    song_input = song_input.lower()
    top_100_check = popvortex_100[popvortex_100['title'] == song_input]
    artist_input=''
    
    if len(top_100_check['title']) > 0 :
        i = randint(1,100)
        while popvortex_100['title'][i] == song_input:   # used to avoid advising the same song as the input
            i = randint(1,100)
        proposed_title = popvortex_100['title'][i]
        proposed_artist = popvortex_100['artist'][i]
        output = 'Great choice ! This song is in the PopVortex Top 100. You should listen to "' + proposed_title + '" by ' + proposed_artist + '. You might like it !'
    
    else :
        res = sp.search(q='track:' + song_input, type='track',limit = 50) 
        total_songs = res['tracks']['total']
        track_id = None
        
        if total_songs > 50:
            artist_input = input('Please fill in the name of a the artist : ')
            artist_input = artist_input.lower()
            res = sp.search(q='track:' + song_input + ' artist:' + artist_input, type='track',limit = 50)

        data = pd.DataFrame(res['tracks']['items'])
        data['name'] = data['name'].apply(lambda x: x.lower())
        data = data[data['name'] == song_input].reset_index(drop = True)
        if len(data['artists']) == 0:
             output = 'We are sorry, but we cannot find any song with this name. Please check your spelling or try another song.'
        else:
            if len(data['artists'])>1:
                if artist_input == '':
                    artist_input = input('Please fill in the name of the artist : ')
                    artist_input = artist_input.lower()
                artist_match = False
                for i in range(len(data['artists'])):
                    art = data['artists'][i][0]['name'].lower()                    
                    if art == artist_input:
                        track_id = data['id'][i]
                        artist_match=True
                        break
                if artist_match == False:
                    output = 'We are sorry, but we cannot find any song with this name and artist. Please check your spelling or try another song.'
                    return output
            else:
                track_id =  data['id'][0]
            track_uri = 'spotify:track:' + track_id
            audio_features = pd.DataFrame(sp.audio_features(track_uri)[0], index=[0])[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
#             audio_features = audio_features[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
            audio_feat_prep = scaler.transform(audio_features)
            song_input_cluster = cluster_model.predict(audio_feat_prep)[0]
            suggestion_list = suggestion_database[suggestion_database['cluster'] == song_input_cluster].reset_index(drop=True)
            j = randint(1,len(suggestion_list['track_name']))
            while suggestion_list['track_name'][j] == song_input:  #used to avoid advising the same song as the input (if ever in the suggestion list)
                j = randint(1,len(suggestion_list['track_name']))
            proposed_title = suggestion_list['track_name'][j]
            proposed_artist = suggestion_list['artist_name'][j]
            output = 'Your song is not in the PopVortex Top 100 but we still have a great advice for you : "' + proposed_title + '" by ' + proposed_artist + '. You might like it !'

    return output
#    return data

In [59]:
song_advice()

Please fill in the name of a song you like :restless river
Please fill in the name of the artist :the magic numbers


'Your song is not in the PopVortex Top 100. Our advice for you is: "Favorite Time Of Year"" by Joe Sample. You might like it !'

In [51]:
song_advice()

Please fill in the name of a song you like :où va le monde
Please fill in the name of a the artist :la femme


"Your song is not in the PopVortex Top 100. Our advice for you is: You're Welcome by Dwayne Johnson. You might like it !"

In [60]:
song_advice()

Please fill in the name of a song you like :swing sahara


'Your song is not in the PopVortex Top 100. Our advice for you is: "Someday"" by Nickelback. You might like it !'

In [61]:
song_advice()

Please fill in the name of a song you like :Poor Lazarus
Please fill in the name of the artist :Bob Dylan


'We are sorry, but we cannot find any song with this name and artist. Please check your spelling or try another song.'